 # \\$sort、\\$limit、\\$skip、\\$count 的基本操作

In [ ]:
# project fields ``numberOfMoons`` and ``name``
db.solarSystem.find({}, {"_id": 0, "name": 1, "numberOfMoons": 1}).pretty()

# count the number of documents
db.solarSystem.find({}, {"_id": 0, "name": 1, "numberOfMoons": 1}).count()

# skip documents
db.solarSystem.find({}, {"_id": 0, "name": 1, "numberOfMoons": 1}).skip(5).pretty()

# limit documents
db.solarSystem.find({}, {"_id": 0, "name": 1, "numberOfMoons": 1}).limit(5).pretty()

# sort documents
db.solarSystem.find({}, { "_id": 0, "name": 1, "numberOfMoons": 1 }).sort( {"numberOfMoons": -1 } ).pretty()

In [ ]:
# ``$limit`` stage
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            numberOfMoons: 1
        }
    },
    { $limit: 5}
]).pretty()

In [ ]:
# ``skip`` stage
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            numberOfMoons: 1
        }
    },
    { $skip: 1 }
]).pretty()

In [ ]:
# ``$count`` stage
db.solarSystem.aggregate([
    {
        $match: {
            type: "Terrestrial planet"
        }
    },
    {
        $project: {
            _id: 0,
            name: 1,
            numberOfMoons: 1
        }
    },
    {
        $count: "terrestrial planets"
    }
]).pretty()

In [ ]:
# removing ``$project`` stage since it does not interfere with our count
db.solarSystem.aggregate([
    {
        $match: {
            type: "Terrestrial planet"
        }
    },
    {
        $count: "terrestrial planets"
    }
]).pretty()

In [ ]:
# ``$sort`` stage
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            numberOfMoons: 1
        }
    },
    { $sort: { numberOfMoons: -1 } }
]).pretty()

In [ ]:
# sorting on more than one field
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            numberOfMoons: 1,
            hasMagneticField: 1
        }
    },
    { $sort: { hasMagneticField: -1, numberOfMoons: -1 } }
]).pretty()

In [ ]:
# setting ``allowDiskUse`` option
# 注意 aggregate RAM 最多使用 100 MB，如果超過就要搭配選項 { allowDiskUse: true } 使用了。 
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            numberOfMoons: 1,
            hasMagneticField: 1
        }
    },
    { $sort: { hasMagneticField: -1, numberOfMoons: -1 } }
], {allowDiskUse: true }).pretty()

### 範例01

In [ ]:
var favorites = [
    "Sandra Bullock",
    "Tom Hanks",
    "Julia Roberts",
    "Kevin Spacey",
    "George Clooney"]

db.movies.aggregate([
    {
        $match: {
            countries: { $in: [ "USA" ] },
            "tomatoes.viewer.rating": { $gte: 3 },
            cast: { $in: favorites }
        }
    },
    {
        $project: {
            _id: 0,
            title: 1,
            tomatoes: "$tomatoes.viewer.rating",
            num_favs: { $size: { $setIntersection: [ "$cast", favorites ] } }
        }
    },
    {
        $sort: {
            num_favs: -1,
            tomatoes: -1,
            title: -1
        }
    },
    { $skip: 24 },
    { $limit: 1}
]).pretty()

### 範例02

In [ ]:
# 這兩個數字怎麼求，我們使用範例03來處理。
x_max = 1521105
x_min = 5

db.movies.aggregate([
    {
        $match: {
            languages: { $in: ["English" ] },
            "imdb.rating": { $gte: 1 },
            "imdb.votes": { $gte: 1},
            year: { $gte: 1990 }
        }
    },
    {
        $project: {
            _id: 0,
            title: 1,
            imdbRating: "$imdb.rating",
            imdbVotes: { $add: [ 1, { $multiply: [ 9, { $divide: [ { $subtract: [ "$imdb.votes", x_min] }, { $subtract: [ x_max, x_min ]  } ] } ] } ]}
        }
    },
    {
        $project: {
            title: 1,
            normalized_rating: { $avg: [ "$imdbRating", "$imdbVotes" ]}
        }
    },
    { $sort: { normalized_rating: 1 }},
    { $limit: 1 }

]).pretty()

### 範例03

In [ ]:
# 要非常注意的是一開始要先進行過濾，排除一些不是數字型態的 documnets。
db.movies.aggregate([
    {
        $match: {
            "imdb.votes": { "$type": "int"} 
        }
    },    
    {
        $group: {
            _id: null,
            max_votes: { $max: "$imdb.votes" },
            min_votes: { $min: "$imdb.votes" }
        }
    }
]).pretty()